This notebook describes the first preprocessing steps of the data. This includes using spacys NER model to recognize organizations, and match these with listed NYSE stocks.

In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'c:\users\cerik\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
# The dataset is too large for git, so I have uploaded it to Google Drive. It can be downloaded using the following snippet:

from google_drive_downloader import GoogleDriveDownloader as gdd
import os

gdd.download_file_from_google_drive(file_id='1SsRqrbMrNgoOQwG-cM3nmPhPGOpzKeP1',
                                    dest_path=os.path.join(os.getcwd(),'reddit.zip'))

In [3]:
# Create data folder structure
import zipfile
import os

if not os.path.exists('data'):
    os.mkdir('data')
    os.mkdir(os.path.join(os.getcwd(), 'data','reddit'))
    os.mkdir(os.path.join(os.getcwd(), 'data','cleaned_reddit'))
    os.mkdir(os.path.join(os.getcwd(), 'data','labeled'))
    os.mkdir(os.path.join(os.getcwd(), 'data','raw_results'))
    os.mkdir(os.path.join(os.getcwd(), 'data','stocks'))
    os.rename('nasdaq_stocks.csv', os.path.join('data','stocks', 'nasdaq_stocks.csv'))
    with zipfile.ZipFile('reddit.zip', 'r') as zip_ref:
        zip_ref.extractall(os.path.join(os.getcwd(), 'data'))

In [6]:
import os

subreddits = os.listdir(os.path.join(os.getcwd(), 'data', 'reddit'))
subreddit_paths = [os.path.join(os.getcwd(), 'data', 'reddit', subreddit, 'submissions_reddit.csv') for subreddit in subreddits]
subreddits


['finance',
 'financialindependence',
 'forex',
 'gme',
 'investing',
 'options',
 'pennystocks',
 'personalfinance',
 'robinhood',
 'robinhoodpennystocks',
 'securityanalysis',
 'stockmarket',
 'stocks',
 'wallstreetbets']

In [8]:
import pandas as pd
df_stocks = pd.read_csv(os.path.join(os.getcwd(), 'data', 'stocks', 'nasdaq_stocks.csv'))
df_stocks = df_stocks[['Symbol', 'Name']]

to_remove = ['Inc', 'Ltd', 'S.A.', 'SE', 'Corp', 'Corporation', 'Incorporated', 
'Class A', 'Common Stock', '(', 'plc', 'PLC', 'Limited','II', 'REIT', 'ADS', 'Co.',
'BDC', 'Enterprise', 'B.H.N.', 'S A', 'INC.', '.com']

def remove_endings(x):
    for rem in to_remove:
        x = x.split(rem)
        x = x[0]
        x = x.strip()
            
    return x

df_stocks['Name'] = df_stocks['Name'].apply(lambda x : remove_endings(x))
df_stocks = df_stocks[~df_stocks['Symbol'].str.contains('^', regex=False)]
df_stocks = df_stocks[~(df_stocks['Name']=='')]

df_stocks.to_csv(os.path.join(os.getcwd(), 'data', 'stocks', 'nasdaq_stocks_clean.csv'))


In [8]:
pd.read_csv(subreddit_paths[0]).columns

Index(['id', 'author', 'created', 'retrieved', 'edited', 'pinned', 'archived',
       'locked', 'removed', 'deleted', 'is_self', 'is_video',
       'is_original_content', 'title', 'link_flair_text', 'upvote_ratio',
       'score', 'gilded', 'total_awards_received', 'num_comments',
       'num_crossposts', 'selftext', 'thumbnail', 'shortlink'],
      dtype='object')

In [9]:
# Combine all csv files into one, 
# removing some unwanted columns and adding a subreddit column
combined_df = pd.DataFrame()
for subreddit, path in zip(subreddits, subreddit_paths):
    df = pd.read_csv(path)
    df = df[['created', 'title', 'selftext', 'upvote_ratio', 'score', 'gilded', 'total_awards_received', 'num_comments']]
    df = df[df['selftext'] != '[removed]']
    df = df[df['selftext'] != '[deleted]']
    df = df.dropna(subset=['selftext', 'title']) # title and selftext is needed for sentiment analysis, so drop those that are empty
    df['stock'] = ''
    df['subreddit'] = subreddit
    combined_df = pd.concat([combined_df, df])

combined_df.to_csv(os.path.join(os.getcwd(), 'data', 'full_combined.csv'))

len(combined_df)

308059

In [15]:
combined_df.head()

,created,title,selftext,upvote_ratio,score,gilded,total_awards_received,num_comments,stock,subreddit
1117,2021-02-24 02:00:17,"Moronic Monday - February 24, 2021 - Your Week...",This is your safe place for questions on finan...,1.00,1,0,0,3,,finance
1192,2021-03-09 02:00:12,"Moronic Monday - March 09, 2021 - Your Weekly ...",This is your safe place for questions on finan...,0.84,4,0,0,7,,finance
1358,2021-03-16 02:00:12,"Moronic Monday - March 16, 2021 - Your Weekly ...",This is your safe place for questions on finan...,1.00,1,0,0,6,,finance
1427,2021-03-23 02:00:20,"Moronic Monday - March 23, 2021 - Your Weekly ...",This is your safe place for questions on finan...,1.00,2,0,0,1,,finance
1488,2021-03-30 02:00:14,"Moronic Monday - March 30, 2021 - Your Weekly ...",This is your safe place for questions on finan...,1.00,1,0,0,5,,finance


In [1]:
import pandas as pd
import os
# aaand load it all again
combined_df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'full_combined.csv'), index_col=0)
df_stocks = pd.read_csv(os.path.join(os.getcwd(), 'data', 'stocks', 'nasdaq_stocks_clean.csv'), index_col=0)

In [10]:
df_stocks.head(50)

,Symbol,Name
0,A,Agilent Technologies
1,AA,Alcoa
2,AAC,Ares Acquisition
3,AACG,ATA Creativity Global American Depositary Shares
4,AACI,Armada Acquisition
5,AACIU,Armada Acquisition
6,AACIW,Armada Acquisition
7,AADI,Aadi Bioscience
8,AAIC,Arlington Asset Investment
11,AAIN,Arlington Asset Investment


In [9]:
len(df_stocks)

7797

In [3]:
len(combined_df)

308059

In [4]:
# create sets of company symbols, and a lookup dict
symbols = set(df_stocks['Symbol'].apply(lambda x : x.upper()))
names = set(df_stocks['Name'].apply(lambda x : x.upper()))
names_to_symbols = {}
for _, (symbol, name) in df_stocks.iterrows():
    names_to_symbols[name.upper()] = symbol.upper()

In [5]:
combined_df.isna().sum()

created                       0
title                         0
selftext                      0
upvote_ratio                  0
score                         0
gilded                        0
total_awards_received         0
num_comments                  0
stock                    308059
subreddit                     0
dtype: int64

In [6]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [7]:
def recognize_stocks(x):
    title = x[1]
    text = x[2]
    title_doc = nlp(title)
    text_doc = nlp(text)
    title_ents = set([ent.text.upper() for ent in title_doc.ents if ent.label_ in ('ORG')])
    text_ents = set([ent.text.upper() for ent in text_doc.ents if ent.label_ in ('ORG')])
    orgs_in_post = title_ents | text_ents # union

    found_symbols = symbols & orgs_in_post # intersection
    found_names = names & orgs_in_post # intersection
    for name in found_names:
        found_symbols.add(names_to_symbols[name])
    
    return found_symbols
mock = (
    '2020', # date
    'Suspicions about buying TsLa stocks', # title
    'I really want to buy Aapl since it will go up, but maybe it will go down. Should I buy them or aDvanced Micro Devices instead?', # text
)

recognize_stocks(mock) # seems to work well enough?

{'AAPL', 'AMD', 'TSLA'}

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

#combined_df.iloc[0:500].progress_apply(lambda x : recognize_stocks(x), axis=1)

combined_df['stock'] = combined_df.progress_apply(lambda x : recognize_stocks(x), axis=1)
combined_df.to_csv(os.path.join(os.getcwd(), 'data', 'full_combined.csv'))

In [23]:
combined_df.tail(20)

,created,title,selftext,upvote_ratio,score,gilded,total_awards_received,num_comments,stock,subreddit
770887,2021-12-23 15:09:58,CROX is getting hammered,I'm not sure that I understand how the traders...,0.87,22,0,0,38,"{CROX, BMO}",wallstreetbets
770895,2021-12-23 15:21:16,"FDA Pfizer, US Army COVID vaccine, Manchin Cou...",Hello everyone again!\n\n1. The FDA has approv...,0.65,8,0,2,20,"{CBOE, PFE}",wallstreetbets
770917,2021-12-23 15:59:26,I'm liking NVAX - the shorts should be getting...,I'm liking NVAX. And the shorts should be gett...,0.88,164,2,22,117,"{NVAX, PFE}",wallstreetbets
771062,2021-12-23 18:29:19,CRNC DD,\n\nhttps://preview.redd.it/zyh7ucac5c781.pn...,0.67,3,0,0,6,"{CRNC, AMZN, AAPL, MSFT}",wallstreetbets
771157,2021-12-23 21:00:13,Weekend Discussion Thread for the Weekend of D...,#HAPPY HOLIDAYS!\n\nWeekend thread has arrived...,0.95,150,0,7,3865,{},wallstreetbets
771177,2021-12-23 21:34:09,Reddit IPO Discussion,&#x200B;\n\n[Publicly Traded Social Media Comp...,0.62,3,0,0,13,{},wallstreetbets
771198,2021-12-23 22:26:21,Markets at close: 12/23/2021,"**Today in brief**\n\n**S&P:** **4,725.79 (+2...",0.87,12,0,0,2,"{SDC, DAL, UAL, MU, PSFE, AAL, AMC}",wallstreetbets
771296,2021-12-24 03:07:24,Buy puts on $LYV Ticketmaster / Live nation.,After the cash burn that happened with this co...,0.94,32,0,0,26,{},wallstreetbets
771332,2021-12-24 04:07:03,KLIC - Kulicke and Soffa Inc is the cross I wi...,"Happy Holidays fellow autists, long time lurke...",0.89,7,0,0,4,"{NDAQ, KLIC}",wallstreetbets
771339,2021-12-24 04:26:02,Higher Natural Gas cost are tailwind for Ammon...,"TTF Natural Gas high of $180, ranging in $140....",0.89,7,0,0,10,"{JWN, NTR, MOS}",wallstreetbets


Next, lets remove the posts that do not talk about any of the NYSE stocks according to our procedure. Some columns appear to contain the word "set()", some contain "{}"

In [1]:
import pandas as pd
combined_df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'full_combined.csv'), index_col=0)
combined_df.tail()

,created,title,selftext,upvote_ratio,score,gilded,total_awards_received,num_comments,stock,subreddit
771454,2021-12-24 15:18:30,NVAX - Lambo or Ramen (Part II),Here's a link to my part I - [LINK](https://ww...,0.92,69,1,3,53,"{'SII', 'NVAX'}",wallstreetbets
771559,2021-12-24 20:01:26,Suspicions about TSLA,Why are people so keen on buying stocks and op...,0.56,19,0,0,214,{'GM'},wallstreetbets
771567,2021-12-24 20:18:09,I’ve Had an Epiphany (Or am I retarted? You te...,I came up with a strategy that seems like a no...,0.80,59,0,0,148,{'ABCL'},wallstreetbets
771580,2021-12-24 21:12:42,Best EV play - Bullish as F ….. MB-private ana...,If I were a TESLA investor... though I'm not.....,0.48,0,0,0,47,"{'TSLA', 'FORD'}",wallstreetbets
771602,2021-12-24 22:22:52,China's Currency Trouble.,The Chinese Yuan is a volatile currency and th...,0.79,68,0,0,68,{'PBC'},wallstreetbets


In [27]:
combined_df[combined_df['stock']=='set()'] = ''
combined_df['stock'] = combined_df['stock'].apply(lambda x : '' if isinstance(x, set) and len(x)==0 else x)
combined_df = combined_df.drop(combined_df[combined_df['stock'] == ''].index)
combined_df.tail()

,created,title,selftext,upvote_ratio,score,gilded,total_awards_received,num_comments,stock,subreddit
771454,2021-12-24 15:18:30,NVAX - Lambo or Ramen (Part II),Here's a link to my part I - [LINK](https://ww...,0.92,69,1,3,53,"{'SII', 'NVAX'}",wallstreetbets
771559,2021-12-24 20:01:26,Suspicions about TSLA,Why are people so keen on buying stocks and op...,0.56,19,0,0,214,{'GM'},wallstreetbets
771567,2021-12-24 20:18:09,I’ve Had an Epiphany (Or am I retarted? You te...,I came up with a strategy that seems like a no...,0.8,59,0,0,148,{'ABCL'},wallstreetbets
771580,2021-12-24 21:12:42,Best EV play - Bullish as F ….. MB-private ana...,If I were a TESLA investor... though I'm not.....,0.48,0,0,0,47,"{'TSLA', 'FORD'}",wallstreetbets
771602,2021-12-24 22:22:52,China's Currency Trouble.,The Chinese Yuan is a volatile currency and th...,0.79,68,0,0,68,{'PBC'},wallstreetbets


In [6]:
combined_df['stock'] = combined_df['stock'].apply(lambda x : x.replace('{','').replace('}','').replace('\'','').replace(' ', ''))
combined_df.tail()

,created,title,selftext,upvote_ratio,score,gilded,total_awards_received,num_comments,stock,subreddit
771454,2021-12-24 15:18:30,NVAX - Lambo or Ramen (Part II),Here's a link to my part I - [LINK](https://ww...,0.92,69,1,3,53,"SII,NVAX",wallstreetbets
771559,2021-12-24 20:01:26,Suspicions about TSLA,Why are people so keen on buying stocks and op...,0.56,19,0,0,214,GM,wallstreetbets
771567,2021-12-24 20:18:09,I’ve Had an Epiphany (Or am I retarted? You te...,I came up with a strategy that seems like a no...,0.80,59,0,0,148,ABCL,wallstreetbets
771580,2021-12-24 21:12:42,Best EV play - Bullish as F ….. MB-private ana...,If I were a TESLA investor... though I'm not.....,0.48,0,0,0,47,"TSLA,FORD",wallstreetbets
771602,2021-12-24 22:22:52,China's Currency Trouble.,The Chinese Yuan is a volatile currency and th...,0.79,68,0,0,68,PBC,wallstreetbets


In [7]:
combined_df.to_csv(os.path.join(os.getcwd(), 'data', 'full_combined.csv'))